In [ ]:
import numpy as np
import controller_utils
import mujoco
from mujoco import viewer
import tools.rotations as rot

# import rospy

import time
time.sleep(0.4)
import mediapy as media

xml_path = 'description/iiwa7_allegro_llm.xml'
obj_name = ''
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)
mujoco.mj_step(model, data)

# view = viewer.launch(model, data)
view = viewer.launch_passive(model, data)
# in notebook, we need ro run view.sync() manually, and set auto_sync=False

obj_names = ['champagne_1', 'champagne_2', 'champagne_3'] # id: 27, 28, 29
num = 0
obj = obj_names[num]
r = controller_utils.Robot(model, data, view, auto_sync=False, obj_names=obj_names)

q0 = np.array([-0.32032434,  0.02706913, -0.22881953, -1.42621454,  1.3862661 , 0.55966738,  1.79477984 - np.pi*3/2]) # home position
r.d.qpos[:7]  = q0 # robot joint angles
r.step()
view.sync()

x0 = np.copy(r.x) # to save the initial quaternion

In [ ]:
import task_planner
task_planner.pick_place_object(model, data, r, view, 'champagne_1', pick=True)
# task_planner.pick_place_object(model, data, r, view, 'champagne_2', pick=True)
# task_planner.pick_place_object(model, data, r, view, 'champagne_3', pick=True)

In [ ]:
from scipy.spatial.transform import Rotation as R

# test KUKA for modifying end effector orientation
test = np.copy(r.x) # Cartesian space, arm, 7

q0 = R.from_quat(test[3:])
q1 = R.from_euler('z', 90, degrees=True)

# Multiply the quaternions to get the combined rotation
q_combined = q0 * q1

# # Get the quaternion as a NumPy array
q = q_combined.as_quat()
# print(q)

test[3:] = q

while True:
    r.iiwa_hand_go(test, r.qh, kh_scale=[0.2,0.2,0.2,0.2])
    view.sync()
    time.sleep(0.002)

In [ ]:
# r.x_obj_dict['champagne_1'] # object pose in Cartesian space
# r.x # end effector pose
# r.p # (4,4) transformation matrix (4, 4) of the end-effector of iiwa

# renderer = mujoco.Renderer(model)
# media.show_image(renderer.render())

# options = mujoco.MjvOption()
# options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True

# mujoco.mj_forward(model, data)
# renderer.update_scene(data)

# media.show_image(renderer.render())

# hand joint configuration
qh = np.array([0, 0.33, 1.05, 1.05, # index
               0, 0.29, 1.2, 0.6, # middle
               0, 0.32, 1.2, 0.8, # ring
               1, 0.5, 0.71,1]) # thumb


In [ ]:
r.modify_joint(q0)
# move the object to a proper pose as an initial pose
p = np.array([0.6,-0.275,1.2])
q = rot.euler2quat([0,0,0])
pose = np.concatenate([p, q])
r.modify_obj_pose(obj, pose)

# move the hand to the grasping configuration

qh = np.array([0, 0.33, 1.05, 1.05, # index
               0, 0.29, 1.2, 0.6, # middle
               0, 0.32, 1.2, 0.8, # ring
               1, 0.5, 0.71,1]) # thumb
r.modify_joint(qh) # assign joints to hand

data.ncon

r.step()
view.sync()

In [ ]:
data.ncon
options = mujoco.MjvOption()
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = 1

In [ ]:
x0 = np.copy(r.x) # Cartesian space, arm, 7
qh0 = np.copy(qh) # Joint position, hand, 16
F_grasp = [0.7, 0.1, 0.3]
while 1:
    # u_add = r.pinch_grasp_force(F_grasp, pairs=[[0,3], [1,3], [2,3]])
    # u_add = r.pinch_grasp_force(F_grasp, pairs=[[0,3]])
    r.iiwa_hand_go(x0, qh0, kh_scale=[0.2,0.2,0.2,0.2])
    view.sync()
    time.sleep(0.002)